In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16357,2024-12-03,Brasil Nbb,19:30,Pato,Flamengo,3.93,1.19,154.5,1.85,1.81,9.5,1.91,1.75,xOp3E9Fa,0.254453,0.840336,0.540541,0.552486,0.094789,0.8,0.4,NaN,NaN,178.244,111.634322,0.626301,96.188,6.453315,0.067091,138.676,103.268,124.46,101.25,0.0,0.0,0.0,0.0,0.756825,0.015456,0.061824,-0.46,-0.092,-31.847826,0.566024,0.5,-0.066024,-0.21,-0.042,-4.523810,0.778232,0.8,0.021768
16358,2024-12-03,Brasil Nbb,20:00,Caxias do Sul,Bauru,2.70,1.38,150.5,1.83,1.83,4.5,2.05,1.66,rLmBCmpC,0.370370,0.724638,0.546448,0.546448,0.095008,0.6,0.8,NaN,NaN,288.336,180.998444,0.627734,120.724,45.649196,0.378129,207.594,183.552,126.28,200.10,0.0,0.0,0.0,0.0,0.457540,0.000000,0.148664,-3.61,-0.722,-2.354571,0.462438,0.2,-0.262438,-2.38,-0.476,-0.798319,0.457220,0.6,0.142780
16359,2024-12-03,Eua Nba,21:00,Charlotte Hornets,Philadelphia 76ers,2.80,1.47,211.5,1.90,1.96,4.5,2.02,1.80,rilAUgY0,0.357143,0.680272,0.526316,0.510204,0.037415,0.2,0.4,NaN,NaN,297.530,89.686621,0.301437,274.424,132.420179,0.482539,261.192,275.700,248.56,237.54,0.0,0.0,0.0,0.0,0.440493,0.021983,0.081447,-3.12,-0.624,-2.884615,0.390131,0.4,0.009869,-2.86,-0.572,-0.821678,0.448052,0.3,-0.148052
16360,2024-12-03,Eua Nba,21:00,Cleveland Cavaliers,Washington Wizards,1.07,10.10,235.5,1.89,1.97,-16.5,2.02,1.79,xjsJSX2D,0.934579,0.099010,0.529101,0.507614,0.033589,0.4,0.4,NaN,NaN,162.140,44.420937,0.273967,606.978,281.571935,0.463892,157.202,644.946,241.50,1026.00,0.0,1.0,0.0,0.0,1.143272,0.029310,0.085372,0.50,0.100,0.700000,0.767511,0.9,0.132489,-5.00,-1.000,-9.100000,0.281603,0.2,-0.081603
16361,2024-12-03,Eua Nba,21:00,Detroit Pistons,Milwaukee Bucks,2.47,1.59,223.5,1.81,1.99,2.5,2.03,1.79,UqU4jyeK,0.404858,0.628931,0.552486,0.502513,0.033789,0.4,0.2,NaN,NaN,318.368,147.586084,0.463571,175.510,36.647338,0.208805,215.422,310.548,168.96,233.20,0.0,0.0,0.0,0.0,0.306529,0.066989,0.088851,0.72,0.144,10.208333,0.447851,0.4,-0.047851,-2.80,-0.560,-1.053571,0.488748,0.2,-0.288748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16483,2024-12-03,Eua Ncaa,22:30,Loyola (Ill).,Eastern Michigan,1.05,12.50,141.5,1.91,1.91,-19.5,2.00,1.71,ppKS27Am,0.952381,0.080000,0.523560,0.523560,0.032381,0.8,0.6,NaN,NaN,109.774,35.287772,0.321458,157.868,55.605718,0.352229,108.834,187.604,85.88,111.74,0.0,0.0,0.0,0.0,1.195037,0.000000,0.110545,1.86,0.372,0.134409,0.711945,0.9,0.188055,7.01,1.402,8.202568,0.351396,0.4,0.048604
16484,2024-12-03,Islândia Premier League Feminina,16:15,Keflavik F,Athena F,1.21,3.74,159.5,1.85,1.81,-12.5,2.01,1.68,Gt8Jzu64,0.826446,0.267380,0.540541,0.552486,0.093826,1.0,0.4,NaN,NaN,138.174,33.218230,0.240409,209.628,97.452202,0.464882,122.692,211.500,115.20,214.20,0.0,0.0,0.0,0.0,0.722820,0.015456,0.126474,1.54,0.308,0.681818,0.000000,0.0,0.000000,-2.98,-0.596,-4.597315,0.000000,0.0,0.000000
16485,2024-12-03,Islândia Premier League Feminina,16:15,Thor AK Akureyri F,Njardvik F,2.20,1.56,159.5,1.86,1.80,1.5,1.83,1.83,lf6RYIzH,0.454545,0.641026,0.537634,0.555556,0.095571,0.8,0.8,NaN,NaN,187.990,62.702705,0.333543,148.666,72.333810,0.486552,149.096,159.372,105.78,257.54,0.0,0.0,0.0,0.0,0.240717,0.023184,0.000000,0.48,0.096,12.500000,0.000000,0.0,0.000000,0.28,0.056,10.000000,0.615227,0.6,-0.015227
16486,2024-12-03,Islândia Premier League Feminina,16:15,Tind

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Charlotte Hornets,Philadelphia 76ers,211.5,1.90,0.6940,Over
1,21:00,Eua Nba,Cleveland Cavaliers,Washington Wizards,235.5,1.89,0.6248,Over
2,22:30,Eua Nba,Dallas Mavericks,Memphis Grizzlies,235.5,1.93,0.9697,Over
3,23:00,Eua Nba,Phoenix Suns,San Antonio Spurs,225.5,1.82,0.7473,Over
4,16:00,Europa Euroliga,Lyon-Villeurbanne,Real Madrid,166.5,1.91,0.7300,Over
5,16:05,Europa Euroliga,Maccabi Tel Aviv,Paris,174.5,1.83,0.6381,Over
6,16:30,Europa Euroliga,Baskonia,Fenerbahce,160.5,1.83,0.6306,Over
